# 01 — Linear and Extended Kalman Filters

> **Level:** Advanced undergraduate / beginning postgraduate

---

## Motivation & Intuition

Kalman Filters provide a recursive algorithm for estimating the hidden state of a (discrete-time) dynamical system when both the **process** and **measurement** noise are Gaussian. Conceptually the filter alternates between two steps:

1. **Predict** — propagate the state distribution through the process model (uncertainty typically increases).
2. **Update** — incorporate a measurement to correct the prior belief (uncertainty typically decreases).

The filter maintains a Gaussian belief $ \mathcal{N}(\hat{x}, P) $. The predict step transforms the mean and covariance under linear dynamics; the update step computes the posterior after incorporating the likelihood.

We will derive the key equations, implement a readable KF, visualize interactive behavior with `ipywidgets`, and then extend to the Extended Kalman Filter (EKF) for nonlinear models.

## Notation and state-space model

We use the discrete-time linear state-space model:

$$
x_k = F_k x_{k-1} + B_k u_k + w_k, \qquad w_k \sim \mathcal{N}(0, Q_k)
$$

$$
z_k = H_k x_k + v_k, \qquad v_k \sim \mathcal{N}(0, R_k)
$$

- $ x_k $: state vector at time $ k $  
- $ u_k $: control input at time $ k $  
- $ z_k $: measurement vector at time $ k $
- $ F_k, B_k, H_k $: system matrices  
- $ Q_k, R_k $ process and measurement covariance matrices

Initial belief: $ x_0 \sim \mathcal{N}(\hat{x}_0, P_0) $.

## Bayesian sketch

From Bayes' theorem:

$$
p(x_k | z_{1:k}) \propto p(z_k | x_k) \, p(x_k | z_{1:k-1}).
$$

Under linear-Gaussian assumptions the prior and likelihood are Gaussians; multiplying two Gaussians yields a Gaussian and closed-form expressions for the posterior mean and covariance. The resulting recursive equations are the Kalman Filter.

## Predict–Update equations 

**Prediction**

$$
\hat{x}_{k|k-1} = F_k \hat{x}_{k-1|k-1} + B_k u_k
$$

$$
P_{k|k-1} = F_k P_{k-1|k-1} F_k^T + Q_k
$$

**Kalman gain**

$$
K_k = P_{k|k-1} H_k^T (H_k P_{k|k-1} H_k^T + R_k)^{-1}
$$

**Update**

$$
\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k (z_k - H_k \hat{x}_{k|k-1})
$$

$$
P_{k|k} = (I - K_k H_k) P_{k|k-1} (I - K_k H_k)^T + K_k R_k K_k^T
$$

(Use the Joseph form above for better numerical stability; the simplified form $ P = (I-KH)P $ is sometimes used.)

### Setting up the environment

In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

## Implementation of `LinearKalmanFilter`

In [2]:
class LinearKalmanFilter:
    """
    Minimal, readable Linear Kalman Filter implementation.

    Parameters
    ----------
    F, H, Q, R : array-like
        System matrices.
    x0 : array-like
        Initial state estimate.
    P0 : array-like
        Initial covariance.
    B : array-like, optional
        Control matrix.
    """

    def __init__(self, F, H, Q, R, x0, P0, B=None):
        self.F = np.atleast_2d(np.array(F, dtype=float))
        self.H = np.atleast_2d(np.array(H, dtype=float))
        self.Q = np.atleast_2d(np.array(Q, dtype=float))
        self.R = np.atleast_2d(np.array(R, dtype=float))
        self.x = np.atleast_1d(np.array(x0, dtype=float))
        self.P = np.atleast_2d(np.array(P0, dtype=float))
        self.B = None if B is None else np.atleast_2d(np.array(B, dtype=float))

    def predict(self, u=None):
        """Predict step: propagate mean and covariance.

        x_{k|k-1} = F x_{k-1|k-1} + B u
        P_{k|k-1} = F P_{k-1|k-1} F^T + Q
        """
        if (self.B is not None) and (u is not None):
            self.x = self.F @ self.x + self.B @ np.atleast_1d(u)
        else:
            self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x, self.P

    def update(self, z):
        """Update step using measurement z.

        K = P H^T (H P H^T + R)^{-1}
        x = x + K (z - H x)
        P = (I - K H) P (I - K H)^T + K R K^T  # Joseph form
        """
        z = np.atleast_1d(z)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ inv(S)
        y = z - (self.H @ self.x)
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ self.H) @ self.P @ (I - K @ self.H).T + K @ self.R @ K.T
        return self.x, self.P, K

    def step(self, z=None, u=None):
        """Convenience: predict then optionally update."""
        self.predict(u=u)
        if z is None:
            return self.x, self.P
        return self.update(z)


### Explanation of the `LinearKalmanFilter` code

- `predict`: computes the prior mean and covariance. If a control input `u` and matrix `B` are provided, it includes the control effect.
- `update`: computes the innovation covariance `S`, the Kalman gain `K`, the innovation `y`, then updates the state and covariance. The Joseph form preserves symmetry and is numerically more robust.
- `step`: a convenience wrapper for predict + optional update.

**Implementation notes**
- Avoid explicit matrix inverse `inv(S)` in performance-critical or large systems — prefer `np.linalg.solve(S, ...)` or Cholesky-based solvers from `scipy.linalg`.
- Check/force symmetry of `P` occasionally: `P = 0.5*(P + P.T)`

## Interactive demo — 1D constant-velocity tracking

This demo simulates a 1D constant-velocity model (state `[position, velocity]`) and noisy position measurements.

Controls:
- `Q_scale`: scales baseline process noise
- `R`: measurement variance

Plots show true trajectory, noisy measurements, KF estimate, and 95% confidence bands.

In [ ]:
# Simulation helper
def simulate_constant_velocity(T=50, dt=1.0, x0=np.array([0.0, 1.0]), Q=None, R=1.0):
    F = np.array([[1, dt],[0,1]])
    H = np.array([[1, 0]])
    if Q is None:
        Q = np.diag([1e-2, 1e-3])
    x = x0.copy().astype(float)
    xs = []
    zs = []
    for k in range(T):
        w = np.random.multivariate_normal(np.zeros(2), Q)
        x = F @ x + w
        v = np.random.normal(0, np.sqrt(R))
        z = float(H @ x + v)
        xs.append(x.copy())
        zs.append(z)
    return np.array(xs), np.array(zs), F, H, Q, R

# Demo runner
def run_demo(Q_scale=1e-2, R=1.0, T=80):
    np.random.seed(1)
    xs, zs, F, H, Q_true, R_true = simulate_constant_velocity(T=T, Q=np.diag([Q_scale, Q_scale*0.5]), R=R)

    x0 = np.array([0.0, 0.0])
    P0 = np.diag([1.0, 1.0])
    kf = LinearKalmanFilter(F=F, H=H, Q=Q_true, R=np.atleast_2d(R_true), x0=x0, P0=P0)

    ests = []
    Ps = []
    for z in zs:
        kf.predict()
        kf.update(z)
        ests.append(kf.x.copy())
        Ps.append(kf.P.copy())
    ests = np.array(ests)
    Ps = np.array(Ps)

    t = np.arange(len(zs))
    fig, ax = plt.subplots(2,1, figsize=(10,6), sharex=True)
    ax[0].plot(t, xs[:,0], '-k', label='true position')
    ax[0].scatter(t, zs, s=10, label='measurements')
    ax[0].plot(t, ests[:,0], '-r', label='KF estimate')
    ax[0].fill_between(t, ests[:,0] - 2*np.sqrt(Ps[:,0,0]), ests[:,0] + 2*np.sqrt(Ps[:,0,0]), alpha=0.2)
    ax[0].legend(); ax[0].set_ylabel('position')

    ax[1].plot(t, xs[:,1], '-k', label='true velocity')
    ax[1].plot(t, ests[:,1], '-r', label='KF estimate')
    ax[1].fill_between(t, ests[:,1] - 2*np.sqrt(Ps[:,1,1]), ests[:,1] + 2*np.sqrt(Ps[:,1,1]), alpha=0.2)
    ax[1].legend(); ax[1].set_ylabel('velocity'); ax[1].set_xlabel('time step')
    plt.show()

# In a Jupyter notebook run the interact cell
interact(run_demo,
         Q_scale=FloatSlider(min=1e-4, max=1e-1, step=1e-4, value=1e-2, description='Q scale'),
         R=FloatSlider(min=1e-3, max=10.0, step=1e-3, value=1.0, description='R'),
         T=IntSlider(min=20, max=200, step=10, value=80));

interactive(children=(FloatSlider(value=0.01, description='Q scale', max=0.1, min=0.0001, step=0.0001), FloatS…

### Explanation of the interactive demo

- `simulate_constant_velocity` builds ground-truth states and noisy position measurements.
- In `run_demo`, we initialize a `LinearKalmanFilter` and run predict/update for each measurement.
- The 95% bands come from ±2 standard deviations from the respective diagonal of `P`.
- Change `Q_scale` and `R` to see filter sensitivity:
  - Larger `R`: measurements are noisy → filter trusts model more, bands widen.
  - Larger `Q`: process uncertainty grows → filter tracks measurements more closely.

## Extended Kalman Filter (EKF)

For nonlinear dynamics:

$$
x_k = f(x_{k-1}, u_k) + w_k, \qquad w_k \sim \mathcal{N}(0,Q_k)
$$

$$
z_k = h(x_k) + v_k, \qquad v_k \sim \mathcal{N}(0,R_k)
$$

Linearize `f` and `h` around current estimate:

$$
F_k = \left.\frac{\partial f}{\partial x}\right|_{\hat{x}_{k-1|k-1}}, \qquad
H_k = \left.\frac{\partial h}{\partial x}\right|_{\hat{x}_{k|k-1}}.
$$

Then apply the KF predict/update using these time-varying Jacobians.

**Algorithm**
1. Predict: $ \hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_k) $, $ P_{k|k-1} = F_k P_{k-1|k-1} F_k^T + Q_k $.
2. Update: compute Kalman gain with $ H_k $, form innovation $ z_k - h(\hat{x}_{k|k-1}) $, update mean and covariance.

**Warning:** EKF can diverge if linearization is poor; use UKF or particle filters for highly nonlinear / multimodal problems.

## Implementation of `ExtendedKalmanFilter`

The inputs used below can be defined as:
- `f(x, u)` (state transition function)
- `h(x)` (measurement function)
- `F_jac(x, u)` (Jacobian of `f` wrt `x`)
- `H_jac(x)` (Jacobian of `h` wrt `x`)


In [4]:
class ExtendedKalmanFilter:
    """
    Simple Extended Kalman Filter implementation.

    User must provide:
      - f(x, u): state transition function
      - h(x): measurement function
      - F_jac(x, u): Jacobian of f wrt x (evaluated at previous/predicted state)
      - H_jac(x): Jacobian of h wrt x (evaluated at predicted state)
    """
    def __init__(self, f, h, F_jac, H_jac, Q, R, x0, P0):
        self.f = f
        self.h = h
        self.F_jac = F_jac
        self.H_jac = H_jac
        self.Q = np.atleast_2d(np.array(Q, dtype=float))
        self.R = np.atleast_2d(np.array(R, dtype=float))
        self.x = np.atleast_1d(np.array(x0, dtype=float))
        self.P = np.atleast_2d(np.array(P0, dtype=float))

    def predict(self, u=None):
        # propagate mean
        self.x = self.f(self.x, u) if u is not None else self.f(self.x, None)
        # compute jacobian and propagate covariance
        F = self.F_jac(self.x, u)
        self.P = F @ self.P @ F.T + self.Q
        return self.x, self.P

    def update(self, z):
        H = self.H_jac(self.x)
        S = H @ self.P @ H.T + self.R
        K = self.P @ H.T @ inv(S)
        y = z - self.h(self.x)
        # If measurements include angles, normalize angle components of y to [-pi, pi]
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ H) @ self.P @ (I - K @ H).T + K @ self.R @ K.T
        return self.x, self.P, K

    def step(self, z=None, u=None):
        self.predict(u=u)
        if z is not None:
            return self.update(z)
        return self.x, self.P

### Explanation of the EKF implementation

- `predict`: uses `f` to propagate the mean and `F_jac` (Jacobian) to propagate covariance.
- `update`: linearizes observation function at the predicted mean (`H_jac`), computes Kalman gain and innovation `y = z - h(x_pred)`, then updates the mean and covariance using Joseph form.
- Remember to normalize angular residuals in `y` when using bearings (wrap to $ [-\pi,\pi] $).


## Interactive demo with a Nonlinear example — Bearing-range tracking (EKF)

We track a target in 2D:

State: $ x = [p_x, p_y, v_x, v_y]^T $ 
Sensor at origin measures bearing and range:

$$
z = \begin{bmatrix} \operatorname{atan2}(p_y, p_x) \\ \sqrt{p_x^2 + p_y^2} \end{bmatrix} + v
$$

Motion model: constant velocity.

We'll simulate the true trajectory with process noise, generate noisy bearing+range measurements, then run the EKF and plot true path vs estimate.

This interactive demo simulates a 2D target moving with **constant velocity** and a sensor at the origin providing **bearing + range** measurements (noisy).  

Controls:
- **Q scale**: multiplies a baseline process-noise covariance (increases process uncertainty / model noise).  
- **Bearing noise (deg)**: standard deviation of bearing measurement noise in degrees.  
- **Range noise (m)**: standard deviation of range measurement noise in meters.  
- **T**: number of time steps to simulate.

What the demo does:
1. Simulates the true trajectory with process noise.
2. Creates noisy bearing+range measurements.
3. Runs the EKF using the supplied `f`, `h`, `F_jac`, `H_jac` (same functions as earlier).
4. Plots: true trajectory (black), EKF estimate (red), measurement-inferred Cartesian points (blue crosses), and the sensor at the origin (green X).  
5. Angle residuals are normalized to $ [-\pi, \pi] $ before update.

Use this to explore how EKF behaves with different noise levels and to observe failure modes (angle wrapping / singularities).

In [6]:
# --- helper: angle normalization ---
def angle_wrap(a):
    """Wrap angle(s) to [-pi, pi]."""
    return (a + np.pi) % (2*np.pi) - np.pi

# --- motion & measurement functions (same as notebook) ---
def f_cv(x, u=None, dt=1.0):
    F = np.array([[1,0,dt,0],
                  [0,1,0,dt],
                  [0,0,1,0],
                  [0,0,0,1]])
    return F @ x

def h_bearing_range(x):
    px, py, vx, vy = x
    rng = np.hypot(px, py)
    bearing = np.arctan2(py, px)
    return np.array([bearing, rng])

F_jac = lambda x, u=None: np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]])

def H_jac(x):
    px, py, vx, vy = x
    rng = np.hypot(px, py)
    if rng < 1e-8:
        return np.zeros((2,4))
    H = np.zeros((2,4))
    H[0,0] = -py / (rng**2)
    H[0,1] = px / (rng**2)
    H[1,0] = px / rng
    H[1,1] = py / rng
    return H

# --- EKF class (lightweight, uses solve for K computation) ---
class ExtendedKalmanFilterSimple:
    def __init__(self, f, h, F_jac, H_jac, Q, R, x0, P0):
        self.f = f
        self.h = h
        self.F_jac = F_jac
        self.H_jac = H_jac
        self.Q = np.atleast_2d(np.array(Q, dtype=float))
        self.R = np.atleast_2d(np.array(R, dtype=float))
        self.x = np.atleast_1d(np.array(x0, dtype=float))
        self.P = np.atleast_2d(np.array(P0, dtype=float))

    def predict(self, u=None):
        self.x = self.f(self.x, u) if u is not None else self.f(self.x, None)
        F = self.F_jac(self.x, u)
        self.P = F @ self.P @ F.T + self.Q
        return self.x, self.P

    def update(self, z):
        H = self.H_jac(self.x)
        S = H @ self.P @ H.T + self.R
        # Solve for K without explicit inverse: K = P H^T S^{-1}
        # compute K by solving S^T y = (P H^T)^T for y, then transpose back
        PHt = self.P @ H.T
        K = np.linalg.solve(S.T, PHt.T).T
        y = z - self.h(self.x)
        # angle normalization (first component is bearing)
        if y.shape[0] >= 1:
            y[0] = angle_wrap(y[0])
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ H) @ self.P @ (I - K @ H).T + K @ self.R @ K.T
        return self.x, self.P, K

    def step(self, z=None, u=None):
        self.predict(u=u)
        if z is not None:
            return self.update(z)
        return self.x, self.P

# --- helper to draw measurement positions from (bearing,range) ---
def meas_to_cart(bearing, rng):
    return rng * np.array([np.cos(bearing), np.sin(bearing)])

# --- interactive runner ---
def run_ekf_demo(Q_scale=1.0, bearing_noise_deg=1.0, range_noise=0.5, T=80, seed=2):
    np.random.seed(seed)
    # Baseline true params
    x_true0 = np.array([5.0, 0.0, 0.5, 0.2])
    Q_base = np.diag([1e-3, 1e-3, 1e-4, 1e-4])
    Q = Q_scale * Q_base
    R = np.diag([np.deg2rad(bearing_noise_deg)**2, range_noise**2])

    # simulate true trajectory and measurements
    xs = []
    zs = []
    x_true = x_true0.copy()
    for k in range(T):
        x_true = f_cv(x_true)
        x_true = x_true + np.random.multivariate_normal(np.zeros(4), Q)
        z = h_bearing_range(x_true) + np.random.multivariate_normal(np.zeros(2), R)
        zs.append(z.copy())
        xs.append(x_true.copy())
    xs = np.array(xs)
    zs = np.array(zs)

    # instantiate EKF
    x0 = np.array([4.0, -1.0, 0.0, 0.0])
    P0 = np.diag([1.0, 1.0, 0.5, 0.5])
    ekf = ExtendedKalmanFilterSimple(f=f_cv, h=h_bearing_range, F_jac=F_jac, H_jac=H_jac, Q=Q, R=R, x0=x0, P0=P0)

    ests = []
    Ps = []
    meas_cart = []
    for z in zs:
        ekf.predict()
        ekf.update(z)
        ests.append(ekf.x.copy())
        Ps.append(ekf.P.copy())
        meas_cart.append(meas_to_cart(z[0], z[1]))
    ests = np.array(ests)
    Ps = np.array(Ps)
    meas_cart = np.array(meas_cart)

    # Plot results
    fig, ax = plt.subplots(1,1, figsize=(8,6))
    ax.plot(xs[:,0], xs[:,1], '-k', lw=1.5, label='true')
    ax.plot(ests[:,0], ests[:,1], '-r', lw=1.5, label='EKF estimate')
    ax.scatter(meas_cart[:,0], meas_cart[:,1], marker='x', c='tab:blue', label='measurements (cartesian)')
    ax.scatter([0], [0], marker='x', c='g', s=80, label='sensor (origin)')
    ax.legend(loc='best')
    ax.set_xlabel('x [m]'); ax.set_ylabel('y [m]'); ax.set_title('Bearing-range tracking (interactive EKF)')
    ax.axis('equal')
    ax.grid(True)

    # optional: draw covariance ellipse for selected times (e.g., last state)
    try:
        cov = Ps[-1][:2,:2]  # position sub-covariance
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        vals = vals[order]
        vecs = vecs[:,order]
        angle = np.degrees(np.arctan2(vecs[1,0], vecs[0,0]))
        width, height = 2*2*np.sqrt(vals)  # 2-sigma ellipse
        ell = Ellipse(xy=(ests[-1,0], ests[-1,1]), width=width, height=height,
                      angle=angle, edgecolor='r', facecolor='none', linestyle='--', linewidth=1.2, label='2-sigma pos')
        ax.add_patch(ell)
    except Exception:
        pass

    plt.show()

# interactive widget controls
interact(run_ekf_demo,
         Q_scale=FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Q scale'),
         bearing_noise_deg=FloatSlider(value=1.0, min=0.01, max=10.0, step=0.1, description='bearing σ (deg)'),
         range_noise=FloatSlider(value=0.5, min=0.01, max=5.0, step=0.01, description='range σ (m)'),
         T=IntSlider(value=80, min=10, max=300, step=10, description='T'),
         seed=IntSlider(value=2, min=0, max=50, step=1, description='seed'));


interactive(children=(FloatSlider(value=1.0, description='Q scale', max=10.0), FloatSlider(value=1.0, descript…

### Explanation and caveats for the bearing-range example

- Bearing is an angular measurement; **always** normalize the angle residual to $ [-\pi, \pi] $ before updating.
- Near sensor singularities (range → 0) the Jacobian becomes ill-conditioned; handle these cases explicitly (fallback, skip update, or use alternate sensor combination).
- EKF uses local linearization — if the posterior uncertainty is large or the system is highly nonlinear, EKF may diverge. UKF or particle filters are alternatives.

## Practical tips, exercises & references

**Practical tips**
- Use the Joseph form for covariance updates to preserve positive semi-definiteness.
- Avoid explicit inverses — prefer solvers (`np.linalg.solve`) or Cholesky (`scipy.linalg.cho_factor` / `cho_solve`).
- Tune `Q` and `R` carefully — they determine filter responsiveness and stability.
- Normalize angle residuals for bearings; symmetrize `P` occasionally.

**Exercises**
1. Implement an Unscented Kalman Filter (UKF) and compare with EKF on the bearing-range example.
2. Implement a square-root Kalman filter and analyze numerical stability.
3. Replace the CV model with a coordinated-turn model and evaluate EKF vs UKF.
4. Fuse IMU + GPS using an EKF and analyze estimate consistency.

**References (cite when quoting)**
- R. E. Kalman, "A New Approach to Linear Filtering and Prediction Problems", *Transactions of the ASME — Journal of Basic Engineering*, 1960.
- G. Welch & G. Bishop, "An Introduction to the Kalman Filter", UNC Chapel Hill, 1995. (tutorial)
- S. J. Julier & J. K. Uhlmann, "A New Extension of the Kalman Filter to Nonlinear Systems", *Proceedings of SPIE*, 1997. (UKF foundational paper)
- D. Simon, *Optimal State Estimation: Kalman, H Infinity, and Nonlinear Approaches*, Wiley, 2006.

---